<a href="https://colab.research.google.com/github/R1tsuko/big_data/blob/main/L1%20-%20Introduction%20to%20Apache%20Spark/BigData_Lab2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving stations.csv to stations.csv


In [ ]:
#
# Лабораторная 1. Неинтерактивный анализ данных велопарковок SF Bay Area Bike Share в Apache Spark
#
# Для каждой стартовой станции найдем среднее время поездки и самую первую поездку.
#
# Команда для запуска
# spark-submit --master yarn --deploy-mode cluster L1_noninteractive_bike_analysis.py
#
# используйте --conf spark.yarn.submit.waitAppCompletion=true, чтобы ждать завершения выполнения спарк задачи
#

from pyspark.sql import SparkSession
from typing import NamedTuple
from datetime import datetime
import numpy as np

spark = SparkSession.builder\
.enableHiveSupport()\
.config('spark.submit.deployMode','cluster')\
.getOrCreate()

sc = spark.sparkContext

tripData = sc.textFile("trips.csv")
# запомним заголовок, чтобы затем его исключить из данных
tripsHeader = tripData.first()
trips = tripData.filter(lambda row: row != tripsHeader).map(lambda row: row.split(",", -1))

stationData = sc.textFile("stations.csv")
stationsHeader = stationData.first()
stations = stationData.filter(lambda row: row != stationsHeader).map(lambda row: row.split(",", -1))

def initStation(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str

    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )

def initTrip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str

    for trip in trips:
        yield Trip(
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M'),
             start_station_name = trip[3],
             start_station_id = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M'),
             end_station_name = trip[6],
             end_station_id = trip[7],
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
        )

stationsInternal = stations.mapPartitions(initStation)
tripsInternal = trips.mapPartitions(initTrip)

tripsByStartStation = tripsInternal.keyBy(lambda trip: trip.start_station_name)

# среднее время поездок стартовой станции

def seqFunc(acc, duration):
    duration_sum, count = acc
    return (duration_sum + duration, count + 1)

def combFunc(acc1, acc2):
    duration_sum1, count1 = acc1
    duration_sum2, count2 = acc2
    return (duration_sum1+duration_sum2, count1+count2)

def meanFunc(acc):
    duration_sum, count = acc
    return duration_sum/count

avgDurationByStartStation = tripsByStartStation\
  .mapValues(lambda trip: trip.duration)\
  .aggregateByKey(
    zeroValue=(0,0),
    seqFunc=seqFunc,
    combFunc=combFunc)\
  .mapValues(meanFunc)\
  .sortBy(lambda x: x[1], ascending=False)

#
# первые поездки стартовых станций
#

firstStationsTrip = tripsByStartStation\
  .reduceByKey(lambda tripA, tripB: tripA if tripA.start_date < tripB.start_date else tripB)

#
# старт выполнения задач и сохранение результата в HDFS
#

avgDurationByStartStation.saveAsTextFile("avg_duration_of_start_stations")
firstStationsTrip.saveAsTextFile("first_station_trips")

# результаты сохранятся в директории распределённой файловой системы в домашней папке пользователя /user/$USER/:
#  - avg_duration_of_start_stations
#  - first_station_trips
#
# получить результаты в виде одного файла можно командой `hadoop fs -getmerge`:
#   hadoop fs -getmerge avg_duration_of_start_stations avg_duration_of_start_stations.txt
#   hadoop fs -getmerge first_station_trips first_station_trips.txt

sc.stop()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (681ee6ac18d4 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/content/spark-3.1.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/content/spark-3.1.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/rdd.py", line -1, in pipeline_func
  File "/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/rdd.py", line -1, in pipeline_func
  File "/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/rdd.py", line 624, in func
    Let p = num / total, where num is the sample size and total is the
  File "/cont